In [1]:
import pandas as pd
import gdown

In [2]:
url = "https://drive.google.com/uc?id=16rfiy-WrqBVBsrmE5VZk-Czk10wMAAmF"
output = "tweets.csv"
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=16rfiy-WrqBVBsrmE5VZk-Czk10wMAAmF
To: /home/onyxia/work/sexism_tweets/tweets.csv
100%|██████████| 1.39M/1.39M [00:00<00:00, 17.6MB/s]


'tweets.csv'

In [3]:
tweets = pd.read_csv('tweets.csv', encoding='utf-8')
tweets.head(10)

,tweet_id,text
0,330425562201993216,NaN
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se..."
3,332500824150392833,NaN
4,334424362033762304,Je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...


# Text preprocessing

### Cleaning text

In [4]:
tweets = tweets.dropna(subset=['text'])

In [5]:
tweets['text_clean'] = tweets["text"].map(lambda x: x.lower() if isinstance(x, str) else x)
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,@mamzellemna une très humble femme! #ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","blogue - «tsé, la parité homme-femme...» au se..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,mise en ligne de mon article sur @payetashnek ...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,achat du jour : le très bon livre #payetashnek...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,hommage #journeedelafemme toute sa vie elle a ...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,prochain concours avec emma on se fait des mèc...
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,#paradiski c'est quoi ? 400 km de pistes verte...
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,mdrrr et avant avec les tél chinois ça disait...


### Removing URLs

In [6]:
import re

url_pattern = re.compile(r'https?://\S+')

def remove_urls(text):
  return url_pattern.sub('', text)


tweets['text_clean'] =  tweets['text_clean'].apply(remove_urls)
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,@mamzellemna une très humble femme! #ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","blogue - «tsé, la parité homme-femme...» au se..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,mise en ligne de mon article sur @payetashnek ...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,achat du jour : le très bon livre #payetashnek...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,hommage #journeedelafemme toute sa vie elle a ...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,prochain concours avec emma on se fait des mèc...
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,#paradiski c'est quoi ? 400 km de pistes verte...
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,mdrrr et avant avec les tél chinois ça disait...


### Replacing emojis and emoticons by their meanings

In [7]:
import emoji
tweets['text_clean'] = tweets['text_clean'].apply(
    lambda x: emoji.replace_emoji(x, replace=lambda c, d: f"[{d['en']}]" ) if isinstance(x, str) else x
)
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,@mamzellemna une très humble femme! #ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","blogue - «tsé, la parité homme-femme...» au se..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,mise en ligne de mon article sur @payetashnek ...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,achat du jour : le très bon livre #payetashnek...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,hommage #journeedelafemme toute sa vie elle a ...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,prochain concours avec emma on se fait des mèc...
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,#paradiski c'est quoi ? 400 km de pistes verte...
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,mdrrr et avant avec les tél chinois ça disait...


In [8]:
from emot.emo_unicode import EMOTICONS_EMO
import re

def replace_emoticons(text):
    if not isinstance(text, str):
        return text
    for emoticon, meaning in EMOTICONS_EMO.items():
        text = re.sub(re.escape(emoticon), f" {meaning} ", text)
    return text

tweets['text_clean'] = tweets['text_clean'].apply(replace_emoticons)
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,@mamzellemna une très humble femme! #ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","blogue - «tsé, la parité homme-femme...» au se..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,mise en ligne de mon article sur @payetashnek ...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,achat du jour : le très bon livre #payetashnek...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,hommage #journeedelafemme toute sa vie elle a ...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,prochain concours avec emma on se fait des mèc...
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,#paradiski c'est quoi ? 400 km de pistes verte...
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,mdrrr et avant avec les tél chinois ça disait...


### Removing non-word and non-whitespace characters

In [9]:
tweets['text_clean'] = tweets['text_clean'].replace(to_replace = r'[^\w\s]', value = '', regex = True)
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,mamzellemna une très humble femme ironie
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...",blogue tsé la parité hommefemme au sein de vi...
4,334424362033762304,Je suis une femme matérialiste et superficiell...,je suis une femme matérialiste et superficiell...
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,mise en ligne de mon article sur payetashnek p...
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,achat du jour le très bon livre payetashnek ...
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,hommage journeedelafemme toute sa vie elle a v...
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,lustre bois flotté feuilles exotiques bleues a...
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,prochain concours avec emma on se fait des mèc...
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,paradiski cest quoi 400 km de pistes vertes b...
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,mdrrr et avant avec les tél chinois ça disait...


### Tokenization

In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

tweets['text_clean'] = tweets['text_clean'].apply(word_tokenize)
tweets.head(10)

[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,"[mamzellemna, une, très, humble, femme, ironie]"
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","[blogue, tsé, la, parité, hommefemme, au, sein..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,"[je, suis, une, femme, matérialiste, et, super..."
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,"[mise, en, ligne, de, mon, article, sur, payet..."
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,"[achat, du, jour, le, très, bon, livre, payeta..."
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,"[hommage, journeedelafemme, toute, sa, vie, el..."
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,"[lustre, bois, flotté, feuilles, exotiques, bl..."
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,"[prochain, concours, avec, emma, on, se, fait,..."
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,"[paradiski, cest, quoi, 400, km, de, pistes, v..."
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,"[mdrrr, et, avant, avec, les, tél, chinois, ça..."


### Removing stopwords

In [11]:
stopwords = [x.strip() for x in open('stop_word_fr.txt').readlines()]
tweets['text_clean'] = tweets['text_clean'].apply(lambda x: [word for word in x if word not in stopwords])
tweets.head(10)

,tweet_id,text,text_clean
1,326796299179548672,@MamzelleMNa Une très humble femme! #Ironie,"[mamzellemna, humble, femme, ironie]"
2,334548844731826176,"BLOGUE - «Tsé, la parité homme-femme...» au se...","[blogue, tsé, parité, hommefemme, vision, sain..."
4,334424362033762304,Je suis une femme matérialiste et superficiell...,"[femme, matérialiste, superficielle, juge, qua..."
5,421708259716636672,Mise en ligne de mon article sur @PayeTaShnek ...,"[mise, ligne, article, payetashnek, deermag, s..."
6,420942263154249728,Achat du jour : le très bon livre #PayeTaShnek...,"[achat, jour, livre, payetashnek, check]"
7,707088385995890689,Hommage #journeedelafemme Toute sa vie elle a ...,"[hommage, journeedelafemme, vie, vendu, fleurs..."
8,955356819324657665,Lustre bois flotté feuilles exotiques bleues a...,"[lustre, bois, flotté, feuilles, exotiques, bl..."
9,955363354083766272,Prochain concours avec Emma on se fait des mèc...,"[prochain, concours, emma, mèches, bleues, met..."
10,955408486153715713,#Paradiski c'est quoi ? 400 km de pistes verte...,"[paradiski, cest, 400, km, pistes, vertes, ble..."
11,955367160334946305,Mdrrr et avant avec les tél chinois ça disait...,"[mdrrr, tél, chinois, disait, carrément, vivre..."
